In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

### Softmax
 - to Convert numbers to probability
 - Since they are probability, they should sum up to 1.

In [3]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis.detach())

tensor([0.0900, 0.2447, 0.6652])


In [4]:
print(hypothesis.sum().detach())

tensor(1.)


In [5]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
print(hypothesis.sum(dim=1))

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)
tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


In [6]:
y = torch.randint(5, (3, )).long()
print(y)

tensor([0, 2, 1])


In [7]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(-1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [8]:
cost = (y_one_hot * - torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [9]:
# Low Level
print(torch.log(F.softmax(z, dim=1)))

# High Level
print(F.log_softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)
tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)


In [10]:
# High Level
## NLL = Negative Log Likelihood
print(F.nll_loss(F.log_softmax(z, dim=1), y).detach())

tensor(1.4689)


In [11]:
# High Level
F.cross_entropy(z, y).detach()

tensor(1.4689)

In [12]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

In [13]:
from torch.utils.data import DataLoader, Dataset

class Data(Dataset):
    def __init__(self):
        self.x_data = torch.FloatTensor(x_train)
        self.y_data = torch.LongTensor(y_train).unsqueeze(-1)
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        
        return x, y

In [14]:
class Model(nn.Module):
    def __init__(self, feature_num, class_num):
        super().__init__()
        self.linear = nn.Linear(feature_num, class_num)
    
    def forward(self, x):
        return self.linear(x)

In [15]:
model = Model(4, 3)
dataset = Data()

In [16]:
dataloader = DataLoader(dataset,
                       batch_size=2,
                       shuffle=True)

In [17]:
optimizer = optim.SGD(model.parameters(), lr=.1)

In [19]:
from IPython.display import clear_output

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    for batch_num, samples in enumerate(dataloader):
        x, y = samples
        y.squeeze_(-1)
        
        hypothesis = model(x)
        cost = F.cross_entropy(hypothesis, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        clear_output(wait=True)
        print("Epoch {:4d}/{} Batch {:2d}/{} Cost: {:.6f}".format(
            epoch, nb_epochs, batch_num+1, len(dataset), cost.item()
        ))

Epoch 1000/1000 Batch  4/8 Cost: 0.009516
